In [1]:
#Packages
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from datetime import datetime
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import BaggingRegressor
from sklearn.ensemble import VotingRegressor
from sklearn import preprocessing
from sklearn.metrics import mean_absolute_error
from statsmodels.tsa.stattools import adfuller
import statsmodels.api as sm
from pylab import rcParams
import math
import time
import xgboost as xgb

In [2]:
#load data, change directory to your corresponding directories
tag = pd.read_csv("C:/Users/cos00/Desktop/Nuclear/APAN5900/Tag Data Final.csv")
#tag = pd.read_csv('~/Desktop/APAN5900/Tag Data Final.csv')

In [3]:
#convert time column from string to time
tag['Time'] = [datetime.strptime(x, '%m/%d/%Y %H:%M:%S') for x in tag['Time'] ] 

In [4]:
#extract tag entries with MI value
tag_MI = tag[tag['MI'].notnull()]

In [5]:
#remove two extreme values that could possibly be outliers
tag_MI = tag_MI[(tag_MI['MI'] != 52.88) & (tag_MI['MI'] != 25.14)]

In [6]:
#--------Model-----------

#split train and test
train, test = train_test_split(tag_MI, test_size=0.2, random_state=42)

In [7]:

#pre-processing, use all numeric predicators
X_train = train.drop(["Time", "ProdGrade", "MI"],axis=1)
Y_train = train["MI"]
X_test = test.drop(["Time", "ProdGrade", "MI"],axis=1)
Y_test = test["MI"]

In [8]:
#model, we chose gbm because it is the best
#rebuild the gradient boosting model with larger number of trees and fix one random state and 20 max features
boosting = GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
             learning_rate=0.1, loss='ls', max_depth=3, max_features=20,
             max_leaf_nodes=None, min_impurity_decrease=0.0,
             min_impurity_split=None, min_samples_leaf=1,
             min_samples_split=2, min_weight_fraction_leaf=0.0,
             n_estimators=500, presort='auto', random_state=115,
             subsample=1.0, verbose=0, warm_start=False)
boosting.fit(X_train, Y_train)
boosting.score(X_train, Y_train)

0.9974520699575301

In [9]:
#predict
MI_gb = boosting.predict(X_test)
#check rmse
mean_squared_error(test['MI'], MI_gb)

0.5838802948664278

In [10]:
#accuracy function
def check_accuracy(prediction):
    pred_round_lm = pd.DataFrame(list(np.round(prediction,1)), test['MI'])
    pred_round_lm.columns = ['predicted']
    accurate = prediction[(pred_round_lm['predicted'] >= pred_round_lm.index * 0.8) & (pred_round_lm['predicted'] <= pred_round_lm.index * 1.2)]
    return accurate.shape[0]/test.shape[0]

In [12]:
#check accuract
check_accuracy(MI_gb)
#92.4% accuracy

0.9244791666666666